In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, SpatialDropout1D
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing import sequence
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from tensorflow.keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.callbacks import Callback


In [ ]:
train = pd.read_csv('/kaggle/input/hsemath2020reviews/train.csv')
test = pd.read_csv('/kaggle/input/hsemath2020reviews/test.csv')

In [ ]:
xtp = train['positive'].values
xtn = train['negative'].values
xp = test['positive'].values
xn = test['negative'].values
y_train = train['score'].values

In [ ]:
max_features = 5000
maxlen = 500
embed_size = 300

tokenizer_p = text.Tokenizer(num_words=max_features)
tokenizer_p.fit_on_texts(list(xtp) + list(xp))
xtp = tokenizer_p.texts_to_sequences(xtp)
xp = tokenizer_p.texts_to_sequences(xp)

tokenizer_n = text.Tokenizer(num_words=max_features)
tokenizer_n.fit_on_texts(list(xtn) + list(xn))
xtn = tokenizer_n.texts_to_sequences(xtn)
xn = tokenizer_n.texts_to_sequences(xn)

In [ ]:
xtp = sequence.pad_sequences(xtp, maxlen=maxlen)
xp = sequence.pad_sequences(xp, maxlen=maxlen)
xtn = sequence.pad_sequences(xtn, maxlen=maxlen)
xn = sequence.pad_sequences(xn, maxlen=maxlen)

In [ ]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

In [ ]:
!unzip -q "wiki-news-300d-1M.vec.zip"

In [ ]:
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open('wiki-news-300d-1M.vec', encoding="utf8"))

In [ ]:
word_index_p = tokenizer_p.word_index
nb_words_p = min(max_features, len(word_index_p))
embedding_matrix_p = np.zeros((nb_words_p, embed_size))
for word, i in word_index_p.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix_p[i] = embedding_vector


In [ ]:
word_index_n = tokenizer_n.word_index
nb_words_n = min(max_features, len(word_index_n))
embedding_matrix_n = np.zeros((nb_words_n, embed_size))
for word, i in word_index_n.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix_n[i] = embedding_vector

In [ ]:
from tensorflow.keras.layers import *

In [ ]:
def build_model(embedding_matrix, lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (maxlen,))
    x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x1 = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(units, return_sequences = True))(x1)
    x = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    
    y = Bidirectional(LSTM(units, return_sequences = True))(x1)
    y = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)
    
    avg_pool1 = GlobalAveragePooling1D()(x)
    max_pool1 = GlobalMaxPooling1D()(x)
    
    avg_pool2 = GlobalAveragePooling1D()(y)
    max_pool2 = GlobalMaxPooling1D()(y)
    
    
    x = concatenate([avg_pool1, max_pool1, avg_pool2, max_pool2])

    # x = Dense(1, activation = "linear")(x)
    model = Model(inputs = inp, outputs = x)
    return model

    
# model = build_model(lr = 1e-3, lr_d = 0, units = 112, dr = 0.2)
# pred = model.predict(test, batch_size = 1024, verbose = 1)


In [ ]:
model1 = build_model(embedding_matrix_p, lr = 1e-3, lr_d = 0, units = 112, dr = 0.2)
model2 = build_model(embedding_matrix_n, lr = 1e-3, lr_d = 0, units = 112, dr = 0.2)

In [ ]:
mergedOut = Add()([model1.output,model2.output])
mergedOut = Flatten()(mergedOut) 
mergedOut = Dropout(.3)(mergedOut)
# mergedOut = Dense(10, activation='relu')(mergedOut)
# # merged = BatchNormalization()(mergedOut)
mergedOut = Dense(256, activation='relu')(mergedOut)
# # merged = BatchNormalization()(mergedOut)
mergedOut = Dense(256, activation='relu')(mergedOut)
# merged = BatchNormalization()(mergedOut)
mergedOut = Dense(256, activation='relu')(mergedOut)
# merged = BatchNormalization()(mergedOut)
mergedOut = Dense(256, activation='elu')(mergedOut)
# # merged = BatchNormalization()(mergedOut)
mergedOut = Dense(256, activation='elu')(mergedOut)
# mergedOut = Dropout(.2)(mergedOut)
mergedOut = Dense(256, activation='relu')(mergedOut)
# # merged = BatchNormalization()(mergedOut)
# mergedOut = Dense(256, activation='relu')(mergedOut)
# # merged = BatchNormalization()(mergedOut)
# mergedOut = Dense(256, activation='relu')(mergedOut)
mergedOut = Dense(1, activation='linear')(mergedOut)

newModel = Model([model1.input,model2.input], mergedOut)
newModel.compile(loss='mean_absolute_error',
                  optimizer='adam')

In [ ]:
callbacks_list = [
 tf.keras.callbacks.EarlyStopping(
 monitor='val_loss',
 patience=3,
 ),
 tf.keras.callbacks.ModelCheckpoint(
 filepath='my_model.h5', 
 monitor='val_loss',
 save_best_only=True,
 ),
 tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                      factor=0.1, 
                                      patience=2)
] 

In [ ]:
newModel.fit([xtp, xtn], y_train, validation_split=0.2, batch_size=64, epochs=20, callbacks=callbacks_list)

In [ ]:
l = newModel.predict([xp, xn])
submission = pd.read_csv('/kaggle/input/hsemath2020reviews/sample_submission(1).csv')
submission['score'] = l
submission.to_csv('sub1.csv', index = False)

In [ ]:
from tensorflow.keras.models import Model, load_model

In [ ]:
model = load_model("my_model.h5")

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
l = newModel.predict([xtp, xtn])

In [ ]:
mean_absolute_error(y_train, l)

In [ ]:
l = model.predict([xp, xn])

In [ ]:
submission = pd.read_csv('/kaggle/input/hsemath2020reviews/sample_submission(1).csv')

In [ ]:
submission['score'] = l

In [ ]:
submission.to_csv('sub2.csv', index = False)